In [1]:
import pandas as pd
import numpy as np
import json
from collections import Counter
from catboost import CatBoostClassifier
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
import itertools
from string import punctuation
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import hstack
import pickle
# from multilabel_data_handler import get_data

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from collections import Counter

In [12]:
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import neighbors
from sklearn import ensemble
from sklearn import neural_network
from sklearn import linear_model
import os

In [13]:
# file used to write preserve the results of the classfier
# confusion matrix and precision recall fscore matrix

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.tight_layout()
    return plt

In [14]:
##saving the classification report
def pandas_classification_report(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred)
    cm = confusion_matrix(y_true, y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    
    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='macro'))
    avg.append(accuracy_score(y_true, y_pred, normalize=True))
    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support','accuracy']
    list_all=list(metrics_summary)
    list_all.append(cm.diagonal())
    class_report_df = pd.DataFrame(
        list_all,
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    total = support.sum() 
    avg[-2] = total

    class_report_df['avg / total'] = avg

    return class_report_df.T

In [15]:
from commen_preprocess import *

In [16]:
eng_train_dataset = pd.read_csv('../AMI@EVALITA2018/en_training.tsv', sep='\t')
eng_test_dataset = pd.read_csv('../AMI@EVALITA2018/en_testing.tsv', sep='\t')
eng_train_dataset = eng_train_dataset.sample(frac=1).reset_index(drop=True)


In [17]:
eng_train_dataset

,id,text,misogynous,misogyny_category,target
0,2643,When your life's lit as FUCK https://t.co/YSSi...,0,0,0
1,3177,When a baby won't stop crying so naturally you...,0,0,0
2,1827,@Oterooooo @Tigraah So you hate 'Ominous' as a...,0,0,0
3,1096,I've said it before and I'll say it again. Don...,1,dominance,active
4,3317,Instead of saying 'that's not all men' how abo...,0,0,0
5,2492,An hour into 'paycheck &; bills' and you reali...,0,0,0
6,2407,@kevinsd73 So true! Yet another case for #Male...,0,0,0
7,3388,@Alyssa_Milano Please take it to the streets A...,0,0,0
8,1279,I could say so much more so keep your skank of...,1,dominance,active
9,1572,@RachaeltheLord I hate when other women drink ...,1,derailing,passive


In [18]:
eng_train_dataset['target'].value_counts()

0          2215
active     1058
passive     727
Name: target, dtype: int64

In [20]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
import os
GLOVE_MODEL_FILE = "../../LEAM-master/glove.twitter.27B/glove.twitter.27B.200d.txt"
# GLOVE_MODEL_FILE="../../../glove.840B.300d.txt"
print(os.path.isfile(GLOVE_MODEL_FILE))




import numpy as np
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    i=0
    for line in f:
        i=i+1
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        model[word] = embedding
        if(i%10000==0):
            print("count"+str(i))
    print("Done.",len(model)," words loaded!")
    return model



EMBEDDING_DIM = 200


# glove_file = GLOVE_MODEL_FILE
# tmp_file = get_tmpfile("test_crawl_300.txt")

# # call glove2word2vec script
# # default way (through CLI): python -m gensim.scripts.glove2word2vec --input <glove_file> --output <w2v_file>
# from gensim.scripts.glove2word2vec import glove2word2vec
# glove2word2vec(glove_file, tmp_file)


# # In[29]:


# word2vec_model = KeyedVectors.load_word2vec_format(tmp_file)
word2vec_model = loadGloveModel(GLOVE_MODEL_FILE)

True
Loading Glove Model
count10000
count20000
count30000
count40000
count50000
count60000
count70000
count80000
count90000
count100000
count110000
count120000
count130000
count140000
count150000
count160000
count170000
count180000
count190000
count200000
count210000
count220000
count230000
count240000
count250000
count260000
count270000
count280000
count290000
count300000
count310000
count320000
count330000
count340000
count350000
count360000
count370000
count380000
count390000
count400000
count410000
count420000
count430000
count440000
count450000
count460000
count470000
count480000
count490000
count500000
count510000
count520000
count530000
count540000
count550000
count560000
count570000
count580000
count590000
count600000
count610000
count620000
count630000
count640000
count650000
count660000
count670000
count680000
count690000
count700000
count710000
count720000
count730000
count740000
count750000
count760000
count770000
count780000
count790000
count800000
count810000
count820000


In [21]:
### stopwords and punctuations are not removed but text is cleaned and stemmed
def glove_tokenize_norem(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    words = text.split()
    words =[ps.stem(word) for word in words]
    return words

####stopwords and punctuations are removed along with that text is cleaned ans stemmed
def glove_tokenize(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    words =[ps.stem(word) for word in words]
    return words
def glove_tokenize_embed(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    return words

In [22]:
def convert_class_label(input_text):
    if input_text==1:
        return 'misogyny'
    elif input_text==0:
        return 'non-misogyny'
    else:
        print('Wrong Input', input_text)
        sys.exit(0)

In [23]:
# pd_train = pd.DataFrame(columns=['id','misogynous','text'])
eng_train_dataset["text"].replace('', np.nan, inplace=True)
eng_train_dataset.dropna(subset=['text'], inplace=True)

pd_train_binary = eng_train_dataset[['id','misogynous','text','misogyny_category','target']]
pd_train_category = eng_train_dataset[['id','misogynous','text','misogyny_category']]
pd_train_target = eng_train_dataset[['id','misogynous','text','target']]
pd_test = eng_test_dataset[['id','text']]

pd_train_category = pd_train_category.loc[pd_train_category['misogynous'] == 1]
pd_train_target = pd_train_target.loc[pd_train_target['misogynous'] == 1]
pd_train_target.drop(['misogynous'], axis=1)                                      
pd_train_category.drop(['misogynous'], axis=1)                                      

# pd_train['class'] =pd_train.apply(lambda row: convert_class_label(row['misogynous']), axis=1)

pd_train_binary['class'] = pd_train_binary['misogynous']
pd_train_category['class'] = pd_train_category['misogyny_category']
pd_train_target['class'] = pd_train_target['target']

# for count, each in enumerate(train_data):
#     try:
#         pd_train.loc[count]  = [each['id'], convert_class_label(each['CounterSpeech']), each['Community'],each['Category'],each['commentText']]
#     except:
#         pass
print('Training Data Loading Completed...')

Training Data Loading Completed...


/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
def get_data(pd_train):
    comments=pd_train['text'].values
    labels=pd_train['class'].values
    list_comment=[]
    for comment,label in zip(comments,labels):
        temp={}
        temp['text']=comment
        temp['label']=label
        list_comment.append(temp)
    return list_comment 

def get_data_test(pd_test):
    comments=pd_test['text'].values
    list_comment=[]
    for comment in comments:
        temp={}
        temp['text']=comment
        list_comment.append(temp)
    return list_comment 


In [25]:
import tensorflow as tf
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
embed = hub.Module(module_url)
config = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=12,
                       allow_soft_placement=True, device_count = {'CPU': 12})

def get_embeddings(messages):
      
    with tf.Session(config=config) as session:
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])
            message_emb = session.run(embed(messages))
            
    print("ending")
    return np.array(message_emb)

/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_10:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_11:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_11
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_12:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/v

In [26]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

TOKENIZER = glove_tokenize
#google encoding used where text is cleaned  
def gen_data_google(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        #X.append(tokenizer(comment['text']))
        X.append(clean(comment['text'], remove_stopwords=True, remove_punctuations=True))
    #TFIDF_feature = 'bpe_text'

    #Word Level Features
    X =get_embeddings(X)
    # print y
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    
    return X, y

#google encoding used where text is not cleaned 
def gen_data_google2(data):
    comments = get_data(data)
    X, y = [],[]
    for comment in comments:
        y.append(comment['label'])
        X.append(clean(comment['text'], remove_stopwords=False, remove_punctuations=False))
    #Word Level Features
    X =get_embeddings(X)
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    return X,y

### tfidf feature generation was used here where stopwords and punctuations are removed 
def gen_data_new_tfidf(data):
    comments = get_data(data)
    comments_test=get_data_test(pd_test)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        X.append(comment['text'])

    X1=[]
    for comment in comments_test:
        X1.append(comment['text'])


    #Word Level Features
    word_vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3),
                min_df=1, 
                strip_accents='unicode',
                #smooth_idf=1,
                analyzer='word', 
                stop_words='english',
                tokenizer=TOKENIZER,             
                max_features=5000)
    
    
    #charlevel features new
    char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    #stop_words='english',
    ngram_range=(2, 6),
    max_features=10000)
    word_vectorizer.fit(X+X1)
    char_vectorizer.fit(X+X1)
    
    with open('tfidf_word_vectorizer.pk', 'wb') as fout:
         pickle.dump(word_vectorizer,fout)

    with open('tfidf_char_vectorizer.pk', 'wb') as fout:
        pickle.dump(char_vectorizer,fout)
    
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    return X, y

### tfidf feature generation was used here where stopwords and punctuations are not removed 
def gen_data_new_tfidf2(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        X.append(comment['text'])


    #Word Level Features
    word_vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3),
                min_df=1, 
                strip_accents='unicode',
                #smooth_idf=1,
                analyzer='word', 
                #stop_words='english',
                tokenizer=glove_tokenize_norem,             
                max_features=5000)
    
    
    #charlevel features new
    char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    #stop_words='english',
    ngram_range=(2, 6),
    max_features=10000)
    
    word_vectorizer.fit(X)
    char_vectorizer.fit(X)
    
    with open('tfidf_word_vectorize_noclean.pk', 'wb') as fout:
         pickle.dump(word_vectorizer,fout)

    with open('tfidf_char_vectorizer_noclean.pk', 'wb') as fout:
         pickle.dump(char_vectorizer,fout)
        
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    
    return X, y

## combination of not cleaned google encodings and tfidf features where stopwords and punctuations are not removed 
def combine_tf_google_rem(data):
    X,_=gen_data_google(data)
    X1,y=gen_data_new_tfidf(data)
#     X1,y=gen_data_old_tfidf()
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

## combination of cleaned google encodings and tfidf features where stopwords and punctuations are ssremoved 
def combine_tf_google_norem(data):
    X,_=gen_data_google2(data)
    X1,y=gen_data_new_tfidf2(data)
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

def combine_tf_rem_google_norem(data):
    X,_=gen_data_google2(data)
    X1,y=gen_data_new_tfidf(data)
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

def combine_tf_norem_google_rem(data):
    X,_=gen_data_google(data)
    X1,y=gen_data_new_tfidf2(data)
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

def gen_data_embed(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        words = glove_tokenize(comment['text'].lower())
        emb = np.zeros(EMBEDDING_DIM)
        for word in words:
            try:
                emb += word2vec_model[word]
            except:
                pass
        if len(words)!=0:
            emb /= len(words)
        X.append(emb)
        y.append(comment['label'])

    return X, y


def combine_tf_rem_google_norem_embed(data):
    X,_=gen_data_google2(data)
    X1,y=gen_data_new_tfidf(data)
    X2,_=gen_data_embed(data)
    X=np.concatenate((np.array(X), np.array(X1),np.array(X2)), axis=1)
    return X,y



###old tfidf

def gen_data_old_tfidf(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        X.append(comment['text'])
    with open('../tfidf_word_vectorizer.pk', 'rb') as fin:
        word_vectorizer = pickle.load(fin)

    with open('../tfidf_char_vectorizer.pk', 'rb') as fin:
        char_vectorizer = pickle.load(fin)


    
    word_vectorizer.fit(X)
    char_vectorizer.fit(X)
    
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    
    return X, y

def select_comments_whose_embedding_exists(flag):
    # selects the comments as in mean_glove_embedding method
    # Processing
    comments = get_data(flag)
    X, Y = [], []
    comment_return = []
    for tweet in comments:
        #print(tweet)
        _emb = 0
        words = TOKENIZER(tweet['text'].lower())
        for w in words:
            #print(w)
            if w in word2vec_model and w is not None:  # Check if embeeding there in GLove model
                _emb+=1
        if _emb:   # Not a blank tweet
            comment_return.append(tweet)
    print('Comments selected:', len(comment_return))
    return comment_return

def gen_data():
    comments = select_comments_whose_embedding_exists(0)
    X, y = [], []
    for comment in comments:
        words = glove_tokenize(comment['text'].lower())
        emb = numpy.zeros(EMBEDDING_DIM)
        for word in words:
            try:
                emb += word2vec_model[word]
            except:
                pass
        emb /= len(words)
        X.append(emb)
        y.append(comment['label'])

    # print y
    y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    print
    return X, y

## combination of not cleaned google encodings and tfidf features where stopwords and punctuations are not removed 
def combine_tf_google_glove_rem():
    X,_=gen_data_google()
    X1,y=gen_data_new_tfidf()
#     X1,y=gen_data_old_tfidf()
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

In [27]:
SCALE_POS_WEIGHT=None

In [28]:
from imblearn.over_sampling import ADASYN
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
def get_model(m_type=None):
    if not m_type:
        print("ERROR: Please specify a model type!")
        return None
    if m_type == 'decision_tree_classifier':
        logreg = tree.DecisionTreeClassifier(class_weight='balanced')
    elif m_type == 'gaussian':
        logreg = GaussianNB()
    elif m_type == 'logistic_regression':
        logreg = LogisticRegression(class_weight='balanced',n_jobs=10, random_state=42)
    elif m_type == 'MLPClassifier':
#         logreg = neural_network.MLPClassifier((500))
        logreg = neural_network.MLPClassifier(random_state=42,early_stopping=True)
    elif m_type == 'KNeighborsClassifier':
#         logreg = neighbors.KNeighborsClassifier(n_neighbors = 10)
        logreg = neighbors.KNeighborsClassifier()
    elif m_type == 'ExtraTreeClassifier':
        logreg = tree.ExtraTreeClassifier()
    elif m_type == 'ExtraTreeClassifier_2':
        logreg = ensemble.ExtraTreesClassifier()
    elif m_type == 'RandomForestClassifier':
        logreg = ensemble.RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=12, max_depth=3)
    elif m_type == 'SVC':
        logreg = LinearSVC(class_weight='balanced');
    elif m_type == 'Catboost':
        logreg = CatBoostClassifier(use_best_model=False, random_state=42, scale_pos_weight=0.8)
#         logreg = CatBoostClassifier(scale_pos_weight=0.8, random_seed=42,);
    elif m_type == 'XGB_classifier':
#         logreg=XGBClassifier(silent=False,eta=0.1,objective='binary:logistic',max_depth=5,min_child_weight=0,gamma=0.2,subsample=0.8, colsample_bytree = 0.8,scale_pos_weight=1,n_estimators=500,reg_lambda=3,nthread=12)
        logreg=XGBClassifier(silent=False,objective='binary:logistic',scale_pos_weight=0.8,reg_lambda=3,nthread=12, random_state=42)
    elif m_type == 'binny_test':
        clf1 = ensemble.RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=12, max_depth=6,max_features='auto')
        clf2 = tree.DecisionTreeClassifier(random_state=42, class_weight='balanced',max_depth=6)
        clf3 = LogisticRegression(class_weight='balanced',penalty="l2",C=0.1, dual=True, random_state=42, n_jobs=3)
        clf4 = XGBClassifier(silent=False,objective='binary:logistic',scale_pos_weight=0.8,reg_lambda=3,nthread=12, random_state=42)
        est_list = [('lr', clf1), ('rf', clf2), ('gnb', clf3), ('xgb', clf4)]
        logreg = ensemble.VotingClassifier(est_list,voting='soft',n_jobs=6)
    else:
        print("give correct model")
    print(logreg)
    return logreg

def get_feature(f_type=None,data=None):
    if not f_type:
        print("ERROR: Please specify a model type!")
        return None
    if f_type == 'google_not_preprocess':
        X,y=gen_data_google2(data)
    elif f_type == 'word_to_vec_embed':
        X,y=gen_data_embed(data)
    elif f_type == 'google_preprocess':
        X,y=gen_data_google(data)
    elif f_type == 'tfidf_not_preprocess':
        X,y=gen_data_new_tfidf2(data)
    elif f_type == 'tfidf_preprocess':
        X,y=gen_data_new_tfidf(data)
    elif f_type == 'google_preprocess_tfidf_preprocess':
        X,y=combine_tf_google_rem(data)
    elif f_type == 'google_nopreprocess_tfidf_nopreprocess':
        X,y=combine_tf_google_norem(data)
    elif f_type == 'google_preprocess_tfidf_nopreprocess':
        X,y=combine_tf_norem_google_rem(data)
    elif f_type == 'google_nopreprocess_tfidf_preprocess':
        X,y=combine_tf_rem_google_norem(data)
    elif f_type == 'google_nopreprocess_tfidf_preprocess_embed':
        X,y=combine_tf_rem_google_norem_embed(data)
    else:
        print("give correct feature selection")    
    print(f_type)
    return X,y



In [29]:
# from imblearn.combine import SMOTETomek 
from sklearn.metrics import accuracy_score
import joblib
from sklearn.model_selection import StratifiedKFold as skf

def binny_classifier_run(X,y,model,model_name,label_map,img_name,report_name,save_model=False):
    Classifier_Train_X = np.array(X, copy=False)
    Classifier_Train_Y = y
    
    
    temp=[]
    for data in Classifier_Train_Y:
        temp.append(label_map[data])
#     print(Classifier_Train_Y)
    Classifier_Train_Y=np.array(temp)
    model_featureSelection = SelectFromModel(ensemble.RandomForestClassifier(n_estimators=50, class_weight='balanced', 
                                                                                     n_jobs=12, max_depth=3))
    print('Before Num features=',Classifier_Train_X.shape[1], Counter(Classifier_Train_Y))
    Classifier_Train_X = model_featureSelection.fit_transform(Classifier_Train_X,Classifier_Train_Y)
    print('After Num features=',Classifier_Train_X.shape[1])

    if(save_model==True):
        Classifier=model
        Classifier.fit(Classifier_Train_X, Classifier_Train_Y)
        filename = 'taskB2/'+model_name+'_task_3.joblib.pkl'
        joblib.dump(Classifier, filename, compress=9)
        filename1 = 'taskB2/'+model_name+'_select_features_task3.joblib.pkl'
        joblib.dump(model_featureSelection, filename1, compress=9)
    else:
        kf = skf(n_splits=10)
        y_total_preds=[] 
        y_total=[]
        count=0

        for train_index, test_index in kf.split(Classifier_Train_X,Classifier_Train_Y):
    #         print('cv_fold',count)
    #         print(train_index, test_index)
    #         print(type(Classifier_Train_X), type(Classifier_Train_Y))
            X_train, X_test = Classifier_Train_X[train_index], Classifier_Train_X[test_index]
            y_train, y_test = Classifier_Train_Y[train_index], Classifier_Train_Y[test_index]

        #     model_featureSelection = SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l1",C=0.1, dual=True, random_state=42, n_jobs=1))
            
        #     ada = ADASYN(random_state=42, ratio='minority')
        #     ada = SMOTETomek(random_state=42,n_jobs=4)
        #     X_train, y_train = ada.fit_sample(X_train, y_train)
        #     print('After OVERSAMPLING Num poinst=', Counter(y_train))
            classifier=model 
            classifier.fit(X_train,y_train)
            y_preds = classifier.predict(X_test)
            for ele in y_test:
                y_total.append(ele)
            for ele in y_preds:
                y_total_preds.append(ele)
            y_pred_train = classifier.predict(X_train)
            count=count+1       
            print('accuracy_train:',accuracy_score(y_train, y_pred_train),'accuracy_test:',accuracy_score(y_test, y_preds))
            print('TRAINING:')
            print(classification_report( y_train, y_pred_train ))
            print("TESTING:")
            print(classification_report( y_test, y_preds ))

        report = classification_report( y_total, y_total_preds )
        cm=confusion_matrix(y_total, y_total_preds)
        plt=plot_confusion_matrix(cm,normalize= True,target_names = ['active','passive'],title = "Confusion Matrix")
        plt.savefig('task3'+model_name+'_'+img_name)
        print(model)
        print(report)
        print(accuracy_score(y_total, y_total_preds))
        df_result=pandas_classification_report(y_total,y_total_preds)
        df_result.to_csv('task3'+model_name+'_'+report_name,  sep=',')


In [30]:
classifier_model = 'gaussian'
feature_model = 'google_nopreprocess_tfidf_preprocess_embed'
img_name = 'cm.png'
report_name = 'report.csv'

In [31]:
data_name= 'pd_train_target'

if(data_name=='pd_train_binary'):
    X,y=get_feature(f_type=feature_model,data=pd_train_binary)
    label_map = {
             1: 1,
             0: 0
                 }
elif(data_name=='pd_train_category'):
    X,y=get_feature(f_type=feature_model,data=pd_train_category)
    label_map = {
            'discredit': 0,
            'sexual_harassment': 1,
            'stereotype': 2,
            'dominance': 3,
            'derailing': 4
        }
elif(data_name=='pd_train_target'):
    X,y=get_feature(f_type=feature_model,data=pd_train_target)
    label_map = {
             'active': 1,
             'passive': 0
         }

else:
    print('give correct data')
    

ending
google_nopreprocess_tfidf_preprocess_embed


In [32]:
model=get_model(m_type=classifier_model)

GaussianNB(priors=None)


In [33]:
#list_of_model = ['decision_tree_classifier', 'gaussian', 'logistic_regression', 'MLPClassifier', 'RandomForestClassifier',
#                 'SVC', 'Catboost', 'XGB_classifier']
list_of_model = [ 'Catboost','XGB_classifier']

for each_model in list_of_model:
    model=get_model(m_type=each_model)
    binny_classifier_run(X,y,model,each_model,label_map,img_name,report_name,save_model=True)


Before Num features= 15712 Counter({1: 1058, 0: 727})
After Num features= 250
Learning rate set to 0.01974
0:	learn: 0.6833435	total: 93.8ms	remaining: 1m 33s
1:	learn: 0.6737663	total: 130ms	remaining: 1m 4s
2:	learn: 0.6652936	total: 159ms	remaining: 52.9s
3:	learn: 0.6582794	total: 186ms	remaining: 46.4s
4:	learn: 0.6502234	total: 212ms	remaining: 42.1s
5:	learn: 0.6433636	total: 240ms	remaining: 39.7s
6:	learn: 0.6355436	total: 263ms	remaining: 37.4s
7:	learn: 0.6293047	total: 289ms	remaining: 35.9s
8:	learn: 0.6226024	total: 317ms	remaining: 34.9s
9:	learn: 0.6152546	total: 346ms	remaining: 34.3s
10:	learn: 0.6091701	total: 373ms	remaining: 33.5s
11:	learn: 0.6026647	total: 400ms	remaining: 33s
12:	learn: 0.5958003	total: 427ms	remaining: 32.4s
13:	learn: 0.5901648	total: 451ms	remaining: 31.8s
14:	learn: 0.5845848	total: 477ms	remaining: 31.3s
15:	learn: 0.5784763	total: 506ms	remaining: 31.1s
16:	learn: 0.5723642	total: 535ms	remaining: 30.9s
17:	learn: 0.5666576	total: 561ms	re

159:	learn: 0.3343662	total: 3.93s	remaining: 20.6s
160:	learn: 0.3338358	total: 3.95s	remaining: 20.6s
161:	learn: 0.3332355	total: 3.98s	remaining: 20.6s
162:	learn: 0.3326499	total: 4s	remaining: 20.6s
163:	learn: 0.3319674	total: 4.03s	remaining: 20.5s
164:	learn: 0.3314248	total: 4.05s	remaining: 20.5s
165:	learn: 0.3309582	total: 4.07s	remaining: 20.5s
166:	learn: 0.3302581	total: 4.1s	remaining: 20.4s
167:	learn: 0.3294445	total: 4.12s	remaining: 20.4s
168:	learn: 0.3288913	total: 4.15s	remaining: 20.4s
169:	learn: 0.3281680	total: 4.17s	remaining: 20.4s
170:	learn: 0.3274449	total: 4.2s	remaining: 20.3s
171:	learn: 0.3268390	total: 4.22s	remaining: 20.3s
172:	learn: 0.3261830	total: 4.24s	remaining: 20.3s
173:	learn: 0.3254103	total: 4.26s	remaining: 20.2s
174:	learn: 0.3248113	total: 4.29s	remaining: 20.2s
175:	learn: 0.3244301	total: 4.31s	remaining: 20.2s
176:	learn: 0.3238577	total: 4.33s	remaining: 20.2s
177:	learn: 0.3234851	total: 4.35s	remaining: 20.1s
178:	learn: 0.322

326:	learn: 0.2547749	total: 7.7s	remaining: 15.9s
327:	learn: 0.2544141	total: 7.72s	remaining: 15.8s
328:	learn: 0.2539574	total: 7.74s	remaining: 15.8s
329:	learn: 0.2536017	total: 7.76s	remaining: 15.8s
330:	learn: 0.2531891	total: 7.78s	remaining: 15.7s
331:	learn: 0.2526316	total: 7.81s	remaining: 15.7s
332:	learn: 0.2521496	total: 7.83s	remaining: 15.7s
333:	learn: 0.2519000	total: 7.85s	remaining: 15.7s
334:	learn: 0.2515758	total: 7.87s	remaining: 15.6s
335:	learn: 0.2512452	total: 7.89s	remaining: 15.6s
336:	learn: 0.2509103	total: 7.91s	remaining: 15.6s
337:	learn: 0.2501051	total: 7.94s	remaining: 15.5s
338:	learn: 0.2497710	total: 7.96s	remaining: 15.5s
339:	learn: 0.2494376	total: 7.98s	remaining: 15.5s
340:	learn: 0.2491574	total: 8s	remaining: 15.5s
341:	learn: 0.2488143	total: 8.02s	remaining: 15.4s
342:	learn: 0.2482984	total: 8.04s	remaining: 15.4s
343:	learn: 0.2480331	total: 8.07s	remaining: 15.4s
344:	learn: 0.2475280	total: 8.09s	remaining: 15.4s
345:	learn: 0.24

489:	learn: 0.1979359	total: 11.3s	remaining: 11.7s
490:	learn: 0.1975063	total: 11.3s	remaining: 11.7s
491:	learn: 0.1973262	total: 11.3s	remaining: 11.7s
492:	learn: 0.1971347	total: 11.3s	remaining: 11.6s
493:	learn: 0.1965910	total: 11.3s	remaining: 11.6s
494:	learn: 0.1961941	total: 11.4s	remaining: 11.6s
495:	learn: 0.1960025	total: 11.4s	remaining: 11.6s
496:	learn: 0.1958369	total: 11.4s	remaining: 11.5s
497:	learn: 0.1956215	total: 11.4s	remaining: 11.5s
498:	learn: 0.1954520	total: 11.5s	remaining: 11.5s
499:	learn: 0.1950662	total: 11.5s	remaining: 11.5s
500:	learn: 0.1946796	total: 11.5s	remaining: 11.5s
501:	learn: 0.1945630	total: 11.5s	remaining: 11.4s
502:	learn: 0.1939038	total: 11.5s	remaining: 11.4s
503:	learn: 0.1936067	total: 11.6s	remaining: 11.4s
504:	learn: 0.1931904	total: 11.6s	remaining: 11.4s
505:	learn: 0.1926666	total: 11.6s	remaining: 11.3s
506:	learn: 0.1924496	total: 11.6s	remaining: 11.3s
507:	learn: 0.1920788	total: 11.7s	remaining: 11.3s
508:	learn: 

649:	learn: 0.1535605	total: 14.8s	remaining: 7.96s
650:	learn: 0.1534227	total: 14.8s	remaining: 7.94s
651:	learn: 0.1532889	total: 14.8s	remaining: 7.91s
652:	learn: 0.1529617	total: 14.8s	remaining: 7.89s
653:	learn: 0.1527622	total: 14.9s	remaining: 7.87s
654:	learn: 0.1525141	total: 14.9s	remaining: 7.84s
655:	learn: 0.1523312	total: 14.9s	remaining: 7.82s
656:	learn: 0.1520084	total: 14.9s	remaining: 7.8s
657:	learn: 0.1518490	total: 15s	remaining: 7.77s
658:	learn: 0.1515709	total: 15s	remaining: 7.75s
659:	learn: 0.1512951	total: 15s	remaining: 7.73s
660:	learn: 0.1510841	total: 15s	remaining: 7.71s
661:	learn: 0.1509198	total: 15s	remaining: 7.68s
662:	learn: 0.1506644	total: 15.1s	remaining: 7.66s
663:	learn: 0.1503751	total: 15.1s	remaining: 7.63s
664:	learn: 0.1502831	total: 15.1s	remaining: 7.61s
665:	learn: 0.1500931	total: 15.1s	remaining: 7.59s
666:	learn: 0.1500090	total: 15.1s	remaining: 7.56s
667:	learn: 0.1496723	total: 15.2s	remaining: 7.54s
668:	learn: 0.1494861	t

814:	learn: 0.1212849	total: 18.4s	remaining: 4.17s
815:	learn: 0.1210354	total: 18.4s	remaining: 4.15s
816:	learn: 0.1208435	total: 18.4s	remaining: 4.13s
817:	learn: 0.1206141	total: 18.4s	remaining: 4.1s
818:	learn: 0.1204583	total: 18.5s	remaining: 4.08s
819:	learn: 0.1203888	total: 18.5s	remaining: 4.06s
820:	learn: 0.1202878	total: 18.5s	remaining: 4.03s
821:	learn: 0.1201462	total: 18.5s	remaining: 4.01s
822:	learn: 0.1199414	total: 18.5s	remaining: 3.99s
823:	learn: 0.1198505	total: 18.6s	remaining: 3.96s
824:	learn: 0.1196721	total: 18.6s	remaining: 3.94s
825:	learn: 0.1194521	total: 18.6s	remaining: 3.92s
826:	learn: 0.1191849	total: 18.6s	remaining: 3.9s
827:	learn: 0.1190275	total: 18.7s	remaining: 3.88s
828:	learn: 0.1188004	total: 18.7s	remaining: 3.85s
829:	learn: 0.1187186	total: 18.7s	remaining: 3.83s
830:	learn: 0.1184930	total: 18.7s	remaining: 3.81s
831:	learn: 0.1183618	total: 18.7s	remaining: 3.78s
832:	learn: 0.1181713	total: 18.8s	remaining: 3.76s
833:	learn: 0.

981:	learn: 0.0961011	total: 22.1s	remaining: 405ms
982:	learn: 0.0959597	total: 22.1s	remaining: 382ms
983:	learn: 0.0958246	total: 22.1s	remaining: 360ms
984:	learn: 0.0957399	total: 22.2s	remaining: 337ms
985:	learn: 0.0956420	total: 22.2s	remaining: 315ms
986:	learn: 0.0955897	total: 22.2s	remaining: 292ms
987:	learn: 0.0954587	total: 22.2s	remaining: 270ms
988:	learn: 0.0952728	total: 22.2s	remaining: 247ms
989:	learn: 0.0951720	total: 22.3s	remaining: 225ms
990:	learn: 0.0950604	total: 22.3s	remaining: 202ms
991:	learn: 0.0948306	total: 22.3s	remaining: 180ms
992:	learn: 0.0947401	total: 22.3s	remaining: 157ms
993:	learn: 0.0946353	total: 22.3s	remaining: 135ms
994:	learn: 0.0944649	total: 22.4s	remaining: 112ms
995:	learn: 0.0943508	total: 22.4s	remaining: 89.9ms
996:	learn: 0.0941627	total: 22.4s	remaining: 67.5ms
997:	learn: 0.0940249	total: 22.4s	remaining: 45ms
998:	learn: 0.0939881	total: 22.5s	remaining: 22.5ms
999:	learn: 0.0939557	total: 22.5s	remaining: 0us
XGBClassifie